# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [25]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [26]:
path = r'C:\Users\Usuario\Desktop\gits\lab-df-calculation-and-transformation\data'
orders = pd.read_csv(path+'/'+'Orders.csv')
orders = orders.drop(['Unnamed: 0'], axis=1)
orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [27]:
# Sub 1 we define a variable with the goupby objet of expenditure by customer. Then put it into dataframe mode
customers_expenditure = pd.DataFrame(orders.groupby('CustomerID')['amount_spent'].sum())

In [28]:
# Sub 2 first we find the quantiles.
customers_expenditure.amount_spent.quantile([.75,.95, 1])

0.75      1661.640
0.95      5840.182
1.00    280206.020
Name: amount_spent, dtype: float64

In [29]:
# now we need to break the dataframe into quantiles to see which customers are in each quantile, we can simulataneusly assing labels to it.
labels=["Preferred", "VIP"]

customers_expenditure= pd.qcut(customers_expenditure['amount_spent'],[ .75,.95,1],labels )
#we transform it into a dataframe to bbe able to drop the NAs which are the consumers that  do not belong to neither Preferred nor VIP.
customers_expenditure = pd.DataFrame(customers_expenditure)

In [33]:
print(type(customers_expenditure))
#finally we drop the NAs and we have the clean classification
customers_expenditure = customers_expenditure.dropna(axis=0)
customers_expenditure

<class 'pandas.core.frame.DataFrame'>


,amount_spent
CustomerID,
12346,VIP
12347,Preferred
12348,Preferred
12349,Preferred
12352,Preferred
...,...
18259,Preferred
18260,Preferred
18272,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [50]:
# we'll need to merge this new dataframe with the other one,we'll be able to do it with customerID, but before, we need to change
#the other columns name for clarity.
customers_expenditure = customers_expenditure.rename(columns={'amount_spent':'customer_type'})
merged_orders =customers_expenditure.merge(right=orders, how='left', on='CustomerID')
#now we group by country and customer type and since luckily there aint that many countries we can see it's the UK.
merged_orders.groupby(['Country','customer_type']).aggregate({'customer_type':'count'})

customer_type
Country         customer_type               
Australia       Preferred                130
                VIP                      898
Austria         Preferred                158
                VIP                        0
Belgium         Preferred               1503
                VIP                       54
Canada          Preferred                135
                VIP                        0
Channel Islands Preferred                225
                VIP                      364
Cyprus          Preferred                203
                VIP                      248
Denmark         Preferred                181
                VIP                       36
EIRE            Preferred                161
                VIP                     7077
Finland         Preferred                210
                VIP                      294
France          Preferred               3011
                VIP                     3290
Germany         Preferred               4222
                VIP                     3127
Greece          Preferred                 32
                VIP                        0
Iceland         Preferred                182
                VIP                        0
Israel          Preferred                214
                VIP                        0
Italy           Preferred                507
                VIP                        0
Japan           Preferred                 67
                VIP                      205
Lebanon         Preferred                 45
                VIP                        0
Malta           Preferred                 67
                VIP                        0
Netherlands     Preferred                  0
                VIP                     2080
Norway          Preferred                608
                VIP                      420
Poland          Preferred                149
                VIP                        0
Portugal        Preferred                412
                VIP                      681
Singapore       Preferred                  0
                VIP                      222
Spain           Preferred               1058
                VIP                      511
Sweden          Preferred                 75
                VIP                      198
Switzerland     Preferred                776
                VIP                      594
United Kingdom  Preferred             137450
                VIP                    84185

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [53]:
merged_orders.groupby(['Country']).aggregate({'customer_type':'count'})

,customer_type
Country,
Australia,1028
Austria,158
Belgium,1557
Canada,135
Channel Islands,589
Cyprus,451
Denmark,217
EIRE,7238
Finland,504


In [ ]:
#again the UK wins by far